<a href="https://colab.research.google.com/github/olhacherenkova/Pet_project_1/blob/main/data_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
# Аутентифікація
auth.authenticate_user()

In [6]:
# Створення клієнта для BigQuery
client = bigquery.Client(project="pet-project-1-487608")

In [7]:
# SQL-запит до бази даних
query = """
SELECT
o.order_id,
o.order_purchase_date,
o.order_status,
oi.seller_id,
oi.price,
p.product_name,
p.product_category,
r.review_ratings
FROM `pet-project-1-487608.cosmetic_sales.cosmetics_order_data` o
LEFT JOIN `pet-project-1-487608.cosmetic_sales.cosmetics_order_items_data` oi
ON o.order_id = oi.order_id
LEFT JOIN `pet-project-1-487608.cosmetic_sales.cosmetic_products_data` p
ON oi.product_id = p.product_id
LEFT JOIN `pet-project-1-487608.cosmetic_sales.cosmetics_review_data` r
ON  o.order_id= r.order_id

"""
# Виконання запиту
query_job = client.query(query)
results = query_job.result()
# Перетворення результатів на DataFrame
df = results.to_dataframe()
# Виведення результату
df.head()

,order_id,order_purchase_date,order_status,seller_id,price,product_name,product_category,review_ratings
0,TB00004NW463,2023-02-08,cancelled,coco_industries,249,Lipstick,Cosmetic,NaN
1,TB00004NW471,2023-02-11,cancelled,coco_industries,540,Probiotic capsules,Cosmetic,NaN
2,TB00004NW481,2023-03-02,cancelled,rk_info_world,399,Moisturizing cream,Cosmetic,3.0
3,TB00004NW514,2023-05-02,cancelled,plum,89,Eyeliner,Cosmetic,NaN
4,TB00004NW517,2023-05-02,cancelled,rk_info_world,250,Compact powder,Cosmetic,4.0


In [17]:
# Огляд даних
print(df.shape)
df.info()

(258, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   order_id             258 non-null    object        
 1   order_purchase_date  258 non-null    datetime64[ns]
 2   order_status         258 non-null    object        
 3   seller_id            258 non-null    object        
 4   price                258 non-null    Int64         
 5   product_name         258 non-null    object        
 6   product_category     258 non-null    object        
 7   review_ratings       116 non-null    float64       
dtypes: Int64(1), datetime64[ns](1), float64(1), object(5)
memory usage: 16.5+ KB


Є відсутні значення тільки у стовбці review_ratings, що зумовлено тим, що не всі покупці залишають відгук.

In [12]:
# Типи даних у кожному стовпчику
print(df.dtypes)

order_id                object
order_purchase_date     dbdate
order_status            object
seller_id               object
price                    Int64
product_name            object
product_category        object
review_ratings         float64
dtype: object


Стовбець order_purchase_date має застарілийтип даних, змінимо його на datetime.

In [15]:
#Зміна типу даних
df['order_purchase_date']=pd.to_datetime(df['order_purchase_date'])
print(df.dtypes)

order_id                       object
order_purchase_date    datetime64[ns]
order_status                   object
seller_id                      object
price                           Int64
product_name                   object
product_category               object
review_ratings                float64
dtype: object


In [19]:
# Дослідження даних на дублікати
duplicate_rows = df.duplicated()
print(duplicate_rows)
print(duplicate_rows.sum())

0      False
1      False
2      False
3      False
4      False
       ...  
253    False
254    False
255    False
256    False
257    False
Length: 258, dtype: bool
0


Дублікатів у даних немає.

In [20]:
df.describe()

,order_purchase_date,price,review_ratings
count,258,258.0,116.000000
mean,2023-07-19 07:48:50.232558080,387.062016,3.577586
min,2023-01-02 00:00:00,59.0,1.000000
25%,2023-04-03 00:00:00,249.0,3.000000
50%,2023-08-10 00:00:00,399.0,4.000000
75%,2023-10-31 18:00:00,510.0,5.000000
max,2023-12-31 00:00:00,799.0,5.000000
std,NaN,188.397574,1.326328


Дані за 2023 рік. Максимальна ціна - 799 доларів, мінімальна - 59. Є велика варіативність цін на замовлення.

In [ ]:
# Зберігаємо дані для візуалізації
df.to_csv("visualization_data.csv", index=False)